# Demagnetization Potential in 3D

We study the magnetostatic potential $u$ generated by a uniformly magnetized cube embedded in an air box.
The magnetization is $\mathbf{m} = (0, 0, 1)$ and the governing weak form reads

$$\int_{\Omega_{\text{air}} \cup \Omega_{\text{mag}}} \nabla u \cdot \nabla v \, \mathrm{d}x = \int_{\Omega_{\text{mag}}} \mathbf{m} \cdot \nabla v \, \mathrm{d}x.$$

The left-hand side spans the entire computational domain (magnet + surrounding air). The right-hand side
is restricted to the magnetic region by means of a discontinuous Galerkin (DG0) indicator function.
This notebook mirrors the style of the 1D/2D exercises: critical code portions are left for you to complete.

## Step 1: Imports

We start by importing the required libraries.

In [ ]:
%matplotlib inline
import numpy as np
import skfem as fem
from skfem.helpers import dot, grad

## Step 2: Load the Mesh

The mesh was generated in Gmsh and contains two subdomains:
- `magnetic`: the magnetized cube
- `air`: the surrounding air box

The mesh file lives under `notebooks/files/cube_with_air.msh`.

In [ ]:
mesh = fem.Mesh.load('files/cube_with_air.msh')
print(mesh)
print('Number of vertices:', mesh.p.shape[1])
print('Number of tetrahedra:', mesh.t.shape[1])
print('Available subdomains:', list(mesh.subdomains.keys()))

## Step 3: Function Spaces and DG0 Indicator

- The scalar potential $u$ is approximated with first-order tetrahedral elements (continuous $P^1$).
- To restrict integrals to the magnet we use a piecewise-constant ($P^0$ / DG0) space. Each DOF lives
  on a single tetrahedron, so it naturally represents indicator values.

We mark the magnetic cells with 1 and the air cells with 0. Interpolating this array into the DG0 space
creates the indicator function `sample`.

In [ ]:
V = fem.Basis(mesh, fem.ElementTetP1())
VDG = fem.Basis(mesh, fem.ElementTetP0())
print('Scalar P1 DOFs:', V.N)
print('DG0 DOFs (one per element):', VDG.N)

sample_values = np.zeros(mesh.nelements)
sample_values[mesh.subdomains['magnetic']] = 1.0
sample = VDG.interpolate(sample_values)
print('Indicator statistics -> min:', sample_values.min(), 'max:', sample_values.max())

## Step 4: Magnetization and Weak Forms (Your Turn)

We prescribe the uniform magnetization vector

$$\mathbf{m} = (0, 0, 1).$$

Complete the bilinear and linear forms:
- **Bilinear form** $a(u, v)$: Laplace form $\int \nabla u \cdot \nabla v$ over the full mesh.
- **Linear form** $L(v)$: restricted source $\int \text{sample} \, (\mathbf{m} \cdot \nabla v)$. Use the DG0 indicator
  to zero-out the contribution outside the magnet (remember `sample` already stores 0/1 per cell).

Fill in the placeholders before running the cell.

In [ ]:
m = np.array([0.0, 0.0, 1.0])

# TODO: Implement the bilinear form a(u, v)
@fem.BilinearForm
def a(u, v, w):
    return _______  # replace with dot(grad(u), grad(v))

# TODO: Implement the linear form L(v)
@fem.LinearForm
def L(v, w):
    return _______  # replace with sample * dot(m, grad(v))

## Step 5: Assemble, Apply Boundary Conditions, and Solve

- Assemble the global stiffness matrix and right-hand side.
- Impose homogeneous Dirichlet data on the outer boundary (mimicking decay at infinity) using `condense`.
- Solve for the potential coefficients and store them in `u`.

In [ ]:
A = a.assemble(V)
b = L.assemble(V)
print('System size:', A.shape)

D = V.get_dofs()  # boundary degrees of freedom
u = fem.solve(*fem.condense(A, b, D=D))
print('Solution stats -> min:', np.min(u), 'max:', np.max(u))

## Step 6: Export the Potential

We export the solution for offline visualization (e.g., ParaView).

In [ ]:
mesh.save('result.vtu', point_data={'potential': u})
print('Saved demag potential to result.vtu')

## Step 7: Prepare for Experiments

We will reuse the gradient of $u$ and a vector-valued basis in the optional experiments below.

In [ ]:
u_field = V.interpolate(u)
VV = fem.Basis(mesh, fem.ElementVector(fem.ElementTetP1()))
print('Vector P1 DOFs:', VV.N)

## Analysis and Discussion

- The DG0 indicator cleanly restricts the excitation to the magnet without modifying the mesh topology.
- Homogeneous Dirichlet conditions on the outer air box emulate far-field decay of the potential.
- Observe how the potential concentrates near the magnet faces normal to the magnetization direction.

## Experiments and Further Tasks

### Task 1 – Manual Projection of the Magnetic Field

Compute the stray field $\mathbf{H} = \nabla u$ by projecting the gradient onto the vector-valued $P^1$ space. Recreate the manual projection
performed in the script by following these steps:
1. Assemble the **vector** mass matrix $M = \int \mathbf{w} \cdot \mathbf{v}$ on `VV`.
2. Assemble the right-hand side $f = \int \nabla u \cdot \mathbf{v}$ using `grad(u_field)`.
3. Solve `M * H = f` for the projected coefficients.
4. Save the projected field (e.g., via `mesh.save(..., point_data={'H_manual': H.reshape(-1, 3)})`).

> **Hint:** Implement the bilinear/linear forms analogous to the scalar case, but keep in mind that
> `u_field` is already an interpolated function you can differentiate inside the form. Outline your own
> code in the cell below; only the plan is provided for you.

In [ ]:
# TODO: Manual L2 projection of grad(u) onto VV
# Step 1: define the vector mass form and assemble it
# Step 2: build the RHS using grad(u_field)
# Step 3: solve for the coefficient vector H_manual
# Step 4: export the result for visualization


### Task 2 – Using `Basis.project`

scikit-fem provides a convenience wrapper for $L^2$ projections. Repeat the previous task but now use
`VV.project(grad(u_field))` to obtain the coefficients directly. Compare both fields in ParaView. The
cell below is intentionally left minimal—add the single line call and any post-processing you need.

In [ ]:
# TODO: Use the built-in projector
# H_auto = VV.project(grad(u_field))
# mesh.save('result_project.vtu', point_data={'H_auto': H_auto.reshape(-1, 3)})
